# MQTT DSCP-Based QoS Analysis

Notebook ini untuk menganalisis hasil experiment DSCP-based QoS pada MQTT dengan SDN.

**Input:** CSV file dari `mqtt_metrics_log.csv`

**Output:**
- Summary statistics
- Visualisasi delay, jitter, packet loss
- Comparison anomaly vs normal traffic
- Export gambar untuk paper

## 1. Setup & Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Set style untuk paper
plt.style.use('seaborn-v0_8-paper')
sns.set_palette("husl")

# Figure size untuk paper (IEEE format)
plt.rcParams['figure.figsize'] = (8, 5)
plt.rcParams['font.size'] = 10
plt.rcParams['axes.labelsize'] = 11
plt.rcParams['axes.titlesize'] = 12
plt.rcParams['xtick.labelsize'] = 9
plt.rcParams['ytick.labelsize'] = 9
plt.rcParams['legend.fontsize'] = 9

print("Libraries imported successfully!")

## 2. Load Data

**EDIT PATH INI:**
- Ganti dengan path ke file CSV Anda
- Atau upload file CSV ke directory yang sama dengan notebook ini

In [ ]:
# ========================================
# EDIT PATH INI SESUAI FILE CSV ANDA
# ========================================
csv_file = "mqtt_metrics_log.csv"  # Jika file ada di directory yang sama
# csv_file = "/path/to/your/mqtt_metrics_log.csv"  # Atau gunakan absolute path

# Load data
df = pd.read_csv(csv_file)

# Display basic info
print(f"Total messages: {len(df):,}")
print(f"\nColumns: {df.columns.tolist()}")
print(f"\nData types:\n{df.dtypes}")
print(f"\nFirst 5 rows:")
df.head()

## 3. Data Preparation & Cleaning

In [ ]:
# Check for missing values
print("Missing values:")
print(df.isnull().sum())

# Remove any rows with missing values (if any)
df = df.dropna()

# Convert delay to seconds for easier interpretation
df['delay_sec'] = df['delay_ms'] / 1000

# Calculate receive timestamp if not exists
if 'recv_timestamp' not in df.columns:
    df['recv_timestamp'] = df['timestamp_sent'] + (df['delay_ms'] / 1000)

# Sort by receive timestamp
df = df.sort_values('recv_timestamp').reset_index(drop=True)

# Calculate jitter (variation in delay)
df['jitter_ms'] = df.groupby('type')['delay_ms'].diff().abs()

print(f"\nData after cleaning: {len(df):,} messages")
print(f"\nTraffic distribution:")
print(df['type'].value_counts())

# Split data by traffic type
df_anomaly = df[df['type'] == 'anomaly'].copy()
df_normal = df[df['type'] == 'normal'].copy()

print(f"\nAnomaly messages: {len(df_anomaly):,}")
print(f"Normal messages: {len(df_normal):,}")

## 4. Statistical Summary

In [ ]:
def calculate_stats(df_type, traffic_type):
    """Calculate statistics for a traffic type"""
    
    # Delay statistics
    delay_stats = {
        'Traffic Type': traffic_type,
        'Count': len(df_type),
        'Avg Delay (ms)': df_type['delay_ms'].mean(),
        'Min Delay (ms)': df_type['delay_ms'].min(),
        'Max Delay (ms)': df_type['delay_ms'].max(),
        'Std Delay (ms)': df_type['delay_ms'].std(),
        'Median Delay (ms)': df_type['delay_ms'].median(),
        'P95 Delay (ms)': df_type['delay_ms'].quantile(0.95),
        'P99 Delay (ms)': df_type['delay_ms'].quantile(0.99),
    }
    
    # Jitter statistics
    jitter_stats = {
        'Avg Jitter (ms)': df_type['jitter_ms'].mean(),
        'Max Jitter (ms)': df_type['jitter_ms'].max(),
        'Std Jitter (ms)': df_type['jitter_ms'].std(),
    }
    
    # Packet loss calculation
    max_seq = df_type['seq'].max()
    expected = max_seq + 1
    received = len(df_type['seq'].unique())
    lost = expected - received
    loss_rate = (lost / expected * 100) if expected > 0 else 0
    
    loss_stats = {
        'Expected Msgs': expected,
        'Received Msgs': received,
        'Lost Msgs': lost,
        'Loss Rate (%)': loss_rate
    }
    
    return {**delay_stats, **jitter_stats, **loss_stats}

# Calculate stats for both traffic types
stats_anomaly = calculate_stats(df_anomaly, 'Anomaly (DSCP 46)')
stats_normal = calculate_stats(df_normal, 'Normal (DSCP 0)')

# Create summary dataframe
summary_df = pd.DataFrame([stats_anomaly, stats_normal]).set_index('Traffic Type')

print("="*80)
print("STATISTICAL SUMMARY")
print("="*80)
print(summary_df.to_string())
print("\n")

# Calculate improvement
delay_improvement = ((stats_normal['Avg Delay (ms)'] - stats_anomaly['Avg Delay (ms)']) / 
                     stats_normal['Avg Delay (ms)'] * 100)
delay_ratio = stats_normal['Avg Delay (ms)'] / stats_anomaly['Avg Delay (ms)']

print("="*80)
print("QoS EFFECTIVENESS")
print("="*80)
print(f"Delay Improvement: {delay_improvement:.2f}%")
print(f"Normal traffic is {delay_ratio:.2f}x slower than anomaly traffic")
print(f"Packet Loss Difference: {stats_normal['Loss Rate (%)'] - stats_anomaly['Loss Rate (%)']: .2f}%")

## 5. Visualization 1: Delay Comparison (Box Plot)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Box plot - Full range
data_to_plot = [df_anomaly['delay_ms'], df_normal['delay_ms']]
bp = ax1.boxplot(data_to_plot, labels=['Anomaly\n(DSCP 46)', 'Normal\n(DSCP 0)'],
                 patch_artist=True, showfliers=False)
bp['boxes'][0].set_facecolor('lightblue')
bp['boxes'][1].set_facecolor('lightcoral')

ax1.set_ylabel('Delay (ms)', fontweight='bold')
ax1.set_title('Delay Distribution (Full Range)', fontweight='bold')
ax1.grid(True, alpha=0.3, linestyle='--')

# Box plot - Zoomed (up to P95)
p95_max = max(df_anomaly['delay_ms'].quantile(0.95), df_normal['delay_ms'].quantile(0.95))
bp2 = ax2.boxplot(data_to_plot, labels=['Anomaly\n(DSCP 46)', 'Normal\n(DSCP 0)'],
                  patch_artist=True, showfliers=False)
bp2['boxes'][0].set_facecolor('lightblue')
bp2['boxes'][1].set_facecolor('lightcoral')

ax2.set_ylabel('Delay (ms)', fontweight='bold')
ax2.set_title('Delay Distribution (Zoomed to P95)', fontweight='bold')
ax2.set_ylim([0, p95_max * 1.1])
ax2.grid(True, alpha=0.3, linestyle='--')

plt.tight_layout()
plt.savefig('delay_comparison_boxplot.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Saved: delay_comparison_boxplot.png")

## 6. Visualization 2: Delay Over Time

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))

# Sample data untuk plotting (setiap 100 messages agar tidak terlalu padat)
sample_rate = max(1, len(df) // 1000)

df_anomaly_sample = df_anomaly.iloc[::sample_rate]
df_normal_sample = df_normal.iloc[::sample_rate]

# Normalize timestamp to start from 0
t_start = df['recv_timestamp'].min()
df_anomaly_sample['time_relative'] = df_anomaly_sample['recv_timestamp'] - t_start
df_normal_sample['time_relative'] = df_normal_sample['recv_timestamp'] - t_start

# Plot
ax.scatter(df_anomaly_sample['time_relative'], df_anomaly_sample['delay_ms'], 
           alpha=0.5, s=10, label='Anomaly (DSCP 46)', color='blue')
ax.scatter(df_normal_sample['time_relative'], df_normal_sample['delay_ms'], 
           alpha=0.5, s=10, label='Normal (DSCP 0)', color='red')

# Add moving average lines
window = 50
df_anomaly_sample['delay_ma'] = df_anomaly_sample['delay_ms'].rolling(window=window, center=True).mean()
df_normal_sample['delay_ma'] = df_normal_sample['delay_ms'].rolling(window=window, center=True).mean()

ax.plot(df_anomaly_sample['time_relative'], df_anomaly_sample['delay_ma'], 
        color='darkblue', linewidth=2, label='Anomaly (Moving Avg)')
ax.plot(df_normal_sample['time_relative'], df_normal_sample['delay_ma'], 
        color='darkred', linewidth=2, label='Normal (Moving Avg)')

ax.set_xlabel('Time (seconds)', fontweight='bold')
ax.set_ylabel('Delay (ms)', fontweight='bold')
ax.set_title('End-to-End Delay Over Time', fontweight='bold', fontsize=14)
ax.legend(loc='upper right')
ax.grid(True, alpha=0.3, linestyle='--')

plt.tight_layout()
plt.savefig('delay_over_time.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Saved: delay_over_time.png")

## 7. Visualization 3: CDF (Cumulative Distribution Function)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

# Calculate CDF
def calculate_cdf(data):
    sorted_data = np.sort(data)
    cdf = np.arange(1, len(sorted_data) + 1) / len(sorted_data)
    return sorted_data, cdf

delay_anomaly, cdf_anomaly = calculate_cdf(df_anomaly['delay_ms'].dropna())
delay_normal, cdf_normal = calculate_cdf(df_normal['delay_ms'].dropna())

# Plot CDF
ax.plot(delay_anomaly, cdf_anomaly, linewidth=2, label='Anomaly (DSCP 46)', color='blue')
ax.plot(delay_normal, cdf_normal, linewidth=2, label='Normal (DSCP 0)', color='red')

# Add vertical lines for percentiles
p50_anomaly = df_anomaly['delay_ms'].median()
p95_anomaly = df_anomaly['delay_ms'].quantile(0.95)
p50_normal = df_normal['delay_ms'].median()
p95_normal = df_normal['delay_ms'].quantile(0.95)

ax.axhline(y=0.5, color='gray', linestyle='--', alpha=0.5, linewidth=1)
ax.axhline(y=0.95, color='gray', linestyle='--', alpha=0.5, linewidth=1)

ax.set_xlabel('Delay (ms)', fontweight='bold')
ax.set_ylabel('CDF', fontweight='bold')
ax.set_title('Cumulative Distribution Function of Delay', fontweight='bold', fontsize=14)
ax.legend(loc='lower right')
ax.grid(True, alpha=0.3, linestyle='--')

# Add text annotations
ax.text(p50_anomaly, 0.5, f'  P50: {p50_anomaly:.1f}ms', 
        verticalalignment='bottom', color='blue', fontsize=8)
ax.text(p95_anomaly, 0.95, f'  P95: {p95_anomaly:.1f}ms', 
        verticalalignment='top', color='blue', fontsize=8)

plt.tight_layout()
plt.savefig('delay_cdf.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Saved: delay_cdf.png")

## 8. Visualization 4: Jitter Comparison

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Histogram
ax1.hist(df_anomaly['jitter_ms'].dropna(), bins=50, alpha=0.6, 
         label='Anomaly (DSCP 46)', color='blue', edgecolor='black')
ax1.hist(df_normal['jitter_ms'].dropna(), bins=50, alpha=0.6, 
         label='Normal (DSCP 0)', color='red', edgecolor='black')
ax1.set_xlabel('Jitter (ms)', fontweight='bold')
ax1.set_ylabel('Frequency', fontweight='bold')
ax1.set_title('Jitter Distribution (Histogram)', fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3, linestyle='--')

# Violin plot
data_jitter = [df_anomaly['jitter_ms'].dropna(), df_normal['jitter_ms'].dropna()]
parts = ax2.violinplot(data_jitter, positions=[1, 2], showmeans=True, showmedians=True)

# Color the violin plots
colors = ['lightblue', 'lightcoral']
for pc, color in zip(parts['bodies'], colors):
    pc.set_facecolor(color)
    pc.set_alpha(0.7)

ax2.set_xticks([1, 2])
ax2.set_xticklabels(['Anomaly\n(DSCP 46)', 'Normal\n(DSCP 0)'])
ax2.set_ylabel('Jitter (ms)', fontweight='bold')
ax2.set_title('Jitter Distribution (Violin Plot)', fontweight='bold')
ax2.grid(True, alpha=0.3, linestyle='--')

plt.tight_layout()
plt.savefig('jitter_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Saved: jitter_comparison.png")

## 9. Visualization 5: Summary Bar Chart

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# Prepare data
metrics = ['Avg Delay (ms)', 'Median Delay (ms)', 'P95 Delay (ms)', 'Avg Jitter (ms)']
anomaly_vals = [stats_anomaly[m] for m in metrics]
normal_vals = [stats_normal[m] for m in metrics]

x = np.arange(len(metrics))
width = 0.35

# Plot 1: Average Delay
ax = axes[0, 0]
bars = ax.bar(['Anomaly', 'Normal'], 
              [stats_anomaly['Avg Delay (ms)'], stats_normal['Avg Delay (ms)']],
              color=['lightblue', 'lightcoral'], edgecolor='black')
ax.set_ylabel('Delay (ms)', fontweight='bold')
ax.set_title('Average Delay Comparison', fontweight='bold')
ax.grid(True, alpha=0.3, axis='y', linestyle='--')

# Add value labels on bars
for bar in bars:
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            f'{height:.1f}', ha='center', va='bottom', fontsize=9)

# Plot 2: Packet Loss
ax = axes[0, 1]
bars = ax.bar(['Anomaly', 'Normal'], 
              [stats_anomaly['Loss Rate (%)'], stats_normal['Loss Rate (%)']],
              color=['lightblue', 'lightcoral'], edgecolor='black')
ax.set_ylabel('Packet Loss (%)', fontweight='bold')
ax.set_title('Packet Loss Rate Comparison', fontweight='bold')
ax.grid(True, alpha=0.3, axis='y', linestyle='--')

for bar in bars:
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            f'{height:.2f}%', ha='center', va='bottom', fontsize=9)

# Plot 3: Jitter
ax = axes[1, 0]
bars = ax.bar(['Anomaly', 'Normal'], 
              [stats_anomaly['Avg Jitter (ms)'], stats_normal['Avg Jitter (ms)']],
              color=['lightblue', 'lightcoral'], edgecolor='black')
ax.set_ylabel('Jitter (ms)', fontweight='bold')
ax.set_title('Average Jitter Comparison', fontweight='bold')
ax.grid(True, alpha=0.3, axis='y', linestyle='--')

for bar in bars:
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            f'{height:.1f}', ha='center', va='bottom', fontsize=9)

# Plot 4: Throughput
ax = axes[1, 1]
duration = df['recv_timestamp'].max() - df['recv_timestamp'].min()
throughput_anomaly = len(df_anomaly) / duration
throughput_normal = len(df_normal) / duration

bars = ax.bar(['Anomaly', 'Normal'], 
              [throughput_anomaly, throughput_normal],
              color=['lightblue', 'lightcoral'], edgecolor='black')
ax.set_ylabel('Throughput (msg/s)', fontweight='bold')
ax.set_title('Message Throughput Comparison', fontweight='bold')
ax.grid(True, alpha=0.3, axis='y', linestyle='--')

for bar in bars:
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            f'{height:.1f}', ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.savefig('summary_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Saved: summary_comparison.png")

## 10. Statistical Test: T-Test

In [ ]:
# Perform t-test to check if difference is statistically significant
t_stat, p_value = stats.ttest_ind(df_anomaly['delay_ms'].dropna(), 
                                   df_normal['delay_ms'].dropna())

print("="*80)
print("STATISTICAL SIGNIFICANCE TEST (Independent T-Test)")
print("="*80)
print(f"Null Hypothesis: No difference in delay between anomaly and normal traffic")
print(f"\nT-statistic: {t_stat:.4f}")
print(f"P-value: {p_value:.10f}")

alpha = 0.05
if p_value < alpha:
    print(f"\n✓ Result: REJECT null hypothesis (p < {alpha})")
    print("  → The difference in delay is STATISTICALLY SIGNIFICANT")
    print("  → DSCP priority is EFFECTIVE in differentiating traffic")
else:
    print(f"\n✗ Result: FAIL TO REJECT null hypothesis (p >= {alpha})")
    print("  → The difference is NOT statistically significant")

# Effect size (Cohen's d)
mean_diff = df_normal['delay_ms'].mean() - df_anomaly['delay_ms'].mean()
pooled_std = np.sqrt((df_anomaly['delay_ms'].std()**2 + df_normal['delay_ms'].std()**2) / 2)
cohens_d = mean_diff / pooled_std

print(f"\nEffect Size (Cohen's d): {cohens_d:.4f}")
if abs(cohens_d) < 0.2:
    effect = "Small"
elif abs(cohens_d) < 0.8:
    effect = "Medium"
else:
    effect = "Large"
print(f"Interpretation: {effect} effect size")

## 11. Export Summary for Paper

In [ ]:
# Create formatted table for paper
paper_summary = pd.DataFrame({
    'Metric': [
        'Messages Received',
        'Average Delay (ms)',
        'Median Delay (ms)',
        'P95 Delay (ms)',
        'Std Dev Delay (ms)',
        'Average Jitter (ms)',
        'Packet Loss Rate (%)',
        'Throughput (msg/s)'
    ],
    'Anomaly (DSCP 46)': [
        f"{stats_anomaly['Count']:,}",
        f"{stats_anomaly['Avg Delay (ms)']:.2f}",
        f"{stats_anomaly['Median Delay (ms)']:.2f}",
        f"{stats_anomaly['P95 Delay (ms)']:.2f}",
        f"{stats_anomaly['Std Delay (ms)']:.2f}",
        f"{stats_anomaly['Avg Jitter (ms)']:.2f}",
        f"{stats_anomaly['Loss Rate (%)']:.2f}",
        f"{throughput_anomaly:.2f}"
    ],
    'Normal (DSCP 0)': [
        f"{stats_normal['Count']:,}",
        f"{stats_normal['Avg Delay (ms)']:.2f}",
        f"{stats_normal['Median Delay (ms)']:.2f}",
        f"{stats_normal['P95 Delay (ms)']:.2f}",
        f"{stats_normal['Std Delay (ms)']:.2f}",
        f"{stats_normal['Avg Jitter (ms)']:.2f}",
        f"{stats_normal['Loss Rate (%)']:.2f}",
        f"{throughput_normal:.2f}"
    ],
    'Improvement': [
        '-',
        f"{delay_improvement:.1f}%",
        f"{((stats_normal['Median Delay (ms)'] - stats_anomaly['Median Delay (ms)']) / stats_normal['Median Delay (ms)'] * 100):.1f}%",
        f"{((stats_normal['P95 Delay (ms)'] - stats_anomaly['P95 Delay (ms)']) / stats_normal['P95 Delay (ms)'] * 100):.1f}%",
        '-',
        f"{((stats_normal['Avg Jitter (ms)'] - stats_anomaly['Avg Jitter (ms)']) / stats_normal['Avg Jitter (ms)'] * 100):.1f}%" if stats_normal['Avg Jitter (ms)'] > 0 else '-',
        f"{(stats_normal['Loss Rate (%)'] - stats_anomaly['Loss Rate (%)']):+.2f}%",
        '-'
    ]
})

print("="*100)
print("SUMMARY TABLE FOR PAPER")
print("="*100)
print(paper_summary.to_string(index=False))
print("\n")

# Save to CSV
paper_summary.to_csv('paper_summary_table.csv', index=False)
print("✓ Saved: paper_summary_table.csv")

# Save to LaTeX format
latex_table = paper_summary.to_latex(index=False, escape=False)
with open('paper_summary_table.tex', 'w') as f:
    f.write(latex_table)
print("✓ Saved: paper_summary_table.tex")

## 12. Key Findings Summary

In [ ]:
print("="*100)
print("KEY FINDINGS FOR PAPER")
print("="*100)

print(f"\n1. DELAY IMPROVEMENT:")
print(f"   - Anomaly traffic (DSCP 46) achieves {delay_improvement:.1f}% lower delay compared to normal traffic")
print(f"   - Normal traffic experiences {delay_ratio:.2f}x higher delay than anomaly traffic")
print(f"   - Average delay: Anomaly = {stats_anomaly['Avg Delay (ms)']:.2f} ms, Normal = {stats_normal['Avg Delay (ms)']:.2f} ms")

print(f"\n2. JITTER REDUCTION:")
jitter_improvement = ((stats_normal['Avg Jitter (ms)'] - stats_anomaly['Avg Jitter (ms)']) / 
                      stats_normal['Avg Jitter (ms)'] * 100) if stats_normal['Avg Jitter (ms)'] > 0 else 0
print(f"   - Anomaly traffic shows {jitter_improvement:.1f}% lower jitter")
print(f"   - Average jitter: Anomaly = {stats_anomaly['Avg Jitter (ms)']:.2f} ms, Normal = {stats_normal['Avg Jitter (ms)']:.2f} ms")

print(f"\n3. PACKET LOSS:")
print(f"   - Anomaly traffic: {stats_anomaly['Loss Rate (%)']:.2f}% packet loss")
print(f"   - Normal traffic: {stats_normal['Loss Rate (%)']:.2f}% packet loss")
print(f"   - Difference: {abs(stats_normal['Loss Rate (%)'] - stats_anomaly['Loss Rate (%)']):.2f}%")

print(f"\n4. STATISTICAL SIGNIFICANCE:")
print(f"   - T-test p-value: {p_value:.10f}")
print(f"   - Result: {'Statistically significant' if p_value < 0.05 else 'Not significant'} (α = 0.05)")
print(f"   - Effect size (Cohen's d): {cohens_d:.4f} ({effect})")

print(f"\n5. OVERALL PERFORMANCE:")
print(f"   - Total messages analyzed: {len(df):,}")
print(f"   - Experiment duration: {duration:.2f} seconds")
print(f"   - Total throughput: {len(df)/duration:.2f} msg/s")
print(f"   - Anomaly/Normal ratio: {len(df_anomaly)/len(df_normal):.2f}:1")

print("\n" + "="*100)
print("CONCLUSION:")
print("="*100)
print(f"The DSCP-based QoS framework successfully prioritizes anomaly traffic (DSCP 46) over")
print(f"normal traffic (DSCP 0), achieving {delay_improvement:.1f}% delay improvement with statistical")
print(f"significance (p < 0.05). The framework demonstrates effective traffic differentiation")
print(f"in Software-Defined Networking environment for MQTT-based IoT applications.")
print("="*100)

## 13. Generate All Figures Summary

In [ ]:
import os

print("="*80)
print("GENERATED FILES SUMMARY")
print("="*80)

figures = [
    'delay_comparison_boxplot.png',
    'delay_over_time.png',
    'delay_cdf.png',
    'jitter_comparison.png',
    'summary_comparison.png'
]

data_files = [
    'paper_summary_table.csv',
    'paper_summary_table.tex'
]

print("\nFIGURES (for paper):")
for fig in figures:
    if os.path.exists(fig):
        size = os.path.getsize(fig) / 1024  # KB
        print(f"  ✓ {fig:40s} ({size:.1f} KB)")
    else:
        print(f"  ✗ {fig:40s} (NOT FOUND)")

print("\nDATA FILES (for paper):")
for data in data_files:
    if os.path.exists(data):
        print(f"  ✓ {data}")
    else:
        print(f"  ✗ {data} (NOT FOUND)")

print("\n" + "="*80)
print("ANALYSIS COMPLETE!")
print("="*80)
print("\nAll figures and tables are ready for your paper.")
print("You can use these files directly in LaTeX or Word document.")